In [321]:
def count_hits(pattern, guess):
    patt_dict = {}
    guess_dict = {}
    for i in range(len(pattern)):
        x = patt_dict.get(pattern[i], [])
        x.append(i)
        patt_dict[pattern[i]] = x

    for i in range(len(guess)):    
        x = guess_dict.get(guess[i], [])
        x.append(i)
        guess_dict[guess[i]] = x


    pseudo_hit = hit = 0

    for key,val in patt_dict.items():
        if guess_dict.get(key,[]):
            match = set(guess_dict[key]).intersection(val)
            hit += len(match)
            pseudo_hit += min(len(guess_dict[key]),len(val)) - len(match)
            
    return hit, pseudo_hit

### Testing

In [107]:
pattern = 'CCCD'
guess   = 'DDDC'
hit, pseudo_hit = count_hits(pattern, guess)
print(f'hit: {hit}')
print(f'pseudo_hit: {pseudo_hit}')



hit: 0
pseudo_hit: 2


## Solution I: Eliminating the patterns that do not match with the guess from the list of all possible patterns

In [372]:
def find_pattern(patterns_list):
    first_guess = patterns_list[0]
    ha, pa = count_hits(pattern, first_guess)
    if ha==4: return first_guess, 1
    patterns_list.remove(first_guess)
    count = 1
    while len(patterns_list)!=1:
        # remove non match patterns
        for m in patterns_list:
            hm, pm = count_hits(m, first_guess)
            if not (hm==ha and pm==pa):
                patterns_list.remove(m)

        idx = random.randint(0,len(patterns_list)-1)                
        first_guess = patterns_list[idx]    
        ha, pa = count_hits(pattern, first_guess)
        count += 1

    return first_guess, count    

In [381]:
import random
res = []
for k in range(300):
    patterns_list = [''.join(_) for _ in product('ABCD', repeat=4)]

    patterns_list.remove('AABB')
    patterns_list.remove('BBCC')
    patterns_list.remove('CCDD')
    patterns_list.remove('DDAA')
    patterns_list.insert(0,'DDAA')
    patterns_list.insert(0,'CCDD')
    patterns_list.insert(0,'BBCC')
    patterns_list.insert(0,'AABB')

    idx = random.randint(0,len(patterns_list)-1)                
    pattern = patterns_list[idx]
    ans, count = find_pattern(patterns_list)
    
    res.append(count)

print(f'average number of queries = {sum(res)/len(res)}')

average number of queries = 9.873333333333333


On average solution I takes about 9.5 to 10 guesses, not very good!!

## Solution II: As in I but with the following heuristic:
## The score of a guess is the minimum number of patterns it can eliminate from the set of all possible scores.

In [373]:
# def tmp_f(pattern, guess):
#     x,y = count_hits(pattern, guess)
#     return x+y

def find_pattern_optimized(patterns_list):
    all_codes = patterns_list 
    patterns_list = all_codes
    key = lambda g: max(Counter(count_hits(g, _) for _ in patterns_list).values())
#   key = lambda g: max(Counter(tmp_f(g, _) for _ in patterns_list).values())

    guess = 'AABB'
    count = 0
    while True:
        count += 1
        ha, pa = count_hits(pattern, guess)
        if guess == pattern:
            return count

        for m in patterns_list:
            hm, pm = count_hits(m, guess)
            if not (hm==ha and pm==pa):
                patterns_list.remove(m)

        if len(patterns_list) == 1:
            guess = patterns_list[0]
        else:
            guess = min(all_codes, key=key) 
            # this heuristic selects the guess with the minimum of number of max matches in the list of all possible patterns 

    return count

In [382]:
from random import randint
from collections import Counter
res = []
for k in range(100):
    patterns_list = [''.join(_) for _ in product('ABCD', repeat=4)]

    idx = randint(0,len(patterns_list)-1)                
    pattern = patterns_list[idx]
    count = find_pattern_optimized(patterns_list)
    
    res.append(count)

print(f'average number of queries = {sum(res)/len(res)}')

average number of queries = 8.1


On average solution II takes about 7 to 8 guesses, better but perhaps not the best that can be achieved!!!
Also, this one is slower than the first one.